<a href="https://colab.research.google.com/github/SedaKUL/sedakul.github.io/blob/master/probability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf 
tf.test.gpu_device_name() 

''

In [0]:
pip install tensorflow==1.6

In [0]:
tf.__version__

'1.6.0'

In [0]:
pip install tqdm

In [0]:
pip install sklearn

In [0]:
pip install pandas 

In [0]:
pip install edward  

     |████████████████████████████████| 71kB 2.8MB/s 
  Created wheel for edward: filename=edward-1.3.5-cp36-none-any.whl size=90389 sha256=b7424a33d19554f3cf3290f1b98d44a7af92adbac9aa233e8b795e941ff3b3ea
  Stored in directory: /root/.cache/pip/wheels/ae/61/0c/1f36f3f0c629d1b7a24d042d2c37015a66c091729c95dd8425
Successfully built edward


In [0]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf 
import pandas as pd
import sklearn.preprocessing as preprocessing
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from edward.models import Normal

%matplotlib inline
plt.style.use('ggplot')

 

In [0]:
adult = pd.read_csv('sample_data/adult.csv',sep= ',',names = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
       "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
       "Hours per week", "Country", "Target"])

adult.head()

adult = adult[1:]
adult.shape

(48842, 15)

In [0]:
from pandas import DataFrame
data = adult[["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
           "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
           "Hours per week", "Country", "Target"]]

data2 = pd.get_dummies(data)
 
data_x = data2.loc[:, :'Country_Yugoslavia'].as_matrix().astype(np.float32)
data_y = data2.loc[:, 'Target_<=50K':].as_matrix().astype(np.float32)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


In [0]:
N = 30000
train_x, test_x = data_x[:N], data_x[N:]
train_y, test_y = data_y[:N], data_y[N:]

in_size = train_x.shape[1]
out_size = train_y.shape[1]


# for bayesian neural network
train_y2 = np.argmax(train_y, axis=1)
test_y2 = np.argmax(test_y, axis=1)

in_size = train_x.shape[1]
out_size = train_y.shape[1]

EPOCH_NUM = 50
BATCH_SIZE = 100

In [0]:
import sys
from tqdm import tqdm
import tensorflow as tf

x_ = tf.placeholder(tf.float32, shape=[None, in_size])
y_ = tf.placeholder(tf.float32, shape=[None, out_size])

w = tf.Variable(tf.truncated_normal([in_size, out_size], stddev=0.1), dtype=tf.float32)
b = tf.Variable(tf.constant(0.1, shape=[out_size]), dtype=tf.float32)
y_pre = tf.matmul(x_, w) + b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_pre))
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in tqdm(range(EPOCH_NUM), file=sys.stdout):
    perm = np.random.permutation(N)
    for i in range(0, N, BATCH_SIZE):
        batch_x = train_x[perm[i:i+BATCH_SIZE]]
        batch_y = train_y[perm[i:i+BATCH_SIZE]]
        train_step.run(session=sess, feed_dict={x_: batch_x, y_: batch_y})
    acc = accuracy.eval(session=sess, feed_dict={x_: train_x, y_: train_y})
    test_acc = accuracy.eval(session=sess, feed_dict={x_: test_x, y_: test_y})
    if (epoch+1) % 1 == 0:
        tqdm.write('epoch:\t{}\taccuracy:\t{}\tvaridation accuracy:\t{}'.format(epoch+1, acc, test_acc))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

epoch:	1	accuracy:	0.7644000053405762	varidation accuracy:	0.7640210390090942
  2%|▏         | 1/50 [00:56<46:30, 56.94s/it]

In [0]:
import edward as ed
import sys
from tqdm import tqdm 
from edward.models import Normal, Categorical

x_ = tf.placeholder(tf.float32, shape=(None, in_size))
y_ = tf.placeholder(tf.int32, shape=(BATCH_SIZE))

w = Normal(loc=tf.zeros([in_size, out_size]), scale=tf.ones([in_size, out_size]))
b = Normal(loc=tf.zeros([out_size]), scale=tf.ones([out_size]))
y_pre = Categorical(tf.matmul(x_, w) + b)

qw = Normal(loc=tf.Variable(tf.random_normal([in_size, out_size])), scale=tf.Variable(tf.random_normal([in_size, out_size])))
qb = Normal(loc=tf.Variable(tf.random_normal([out_size])), scale=tf.Variable(tf.random_normal([out_size])))

y = Categorical(tf.matmul(x_, qw) + qb)

inference = ed.KLqp({w: qw, b: qb}, data={y_pre: y_})
inference.initialize()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

with sess:
    samples_num = 100
    for epoch in tqdm(range(EPOCH_NUM), file=sys.stdout):
        perm = np.random.permutation(N)
        for i in range(0, N, BATCH_SIZE):
            batch_x = train_x[perm[i:i+BATCH_SIZE]]
            batch_y = train_y2[perm[i:i+BATCH_SIZE]]
            inference.update(feed_dict={x_: batch_x, y_: batch_y})
        y_samples = y.sample(samples_num).eval(feed_dict={x_: train_x})
        acc = (np.round(y_samples.sum(axis=0) / samples_num) == train_y2).mean()
        y_samples = y.sample(samples_num).eval(feed_dict={x_: test_x})
        test_acc = (np.round(y_samples.sum(axis=0) / samples_num) == test_y2).mean()
        if (epoch+1) % 1 == 0:
            tqdm.write('epoch:\t{}\taccuracy:\t{}\tvaridation accuracy:\t{}'.format(epoch+1, acc, test_acc))

epoch:	1	accuracy:	0.6549	varidation accuracy:	0.7162190850228214
epoch:	2	accuracy:	0.7826	varidation accuracy:	0.7130877826133107
epoch:	3	accuracy:	0.7766666666666666	varidation accuracy:	0.785054665109861
epoch:	4	accuracy:	0.7978333333333333	varidation accuracy:	0.774599299437427
epoch:	5	accuracy:	0.7778	varidation accuracy:	0.7747054452818172
epoch:	6	accuracy:	0.7381666666666666	varidation accuracy:	0.761861798110604
epoch:	7	accuracy:	0.7544	varidation accuracy:	0.8057000318437533
epoch:	8	accuracy:	0.7879666666666667	varidation accuracy:	0.7695573718288929
epoch:	9	accuracy:	0.7918333333333333	varidation accuracy:	0.7886105508969324
epoch:	10	accuracy:	0.7542666666666666	varidation accuracy:	0.7753423203481584
epoch:	11	accuracy:	0.7648333333333334	varidation accuracy:	0.7649400275979196
epoch:	12	accuracy:	0.7543333333333333	varidation accuracy:	0.796624562148392
epoch:	13	accuracy:	0.7890333333333334	varidation accuracy:	0.7041715316845345
epoch:	14	accuracy:	0.7181	varidat